In [1]:
pwd

u'/Users/Tultul/Dropbox/DSI_G/week10/day1'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
from datetime import datetime
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from sklearn.metrics import precision_score, recall_score, roc_curve
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from bs4 import BeautifulSoup
from time import gmtime
%matplotlib inline
import sys
import os
import matplotlib.pyplot as plt
import matplotlib

In [4]:
delinq_df = pd.read_csv('/Users/Tultul/Dropbox/DSI_G/DelinqProject/DelinquencyModel/DelinqData500K.csv')

In [6]:
delinq_df.head()

,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,1,8535032161,1072,8630.000,9191.400,784.48,2007.28,7.0,NaN,773.0,...,6.0,0.0,2,12,6.0,0.0,2.0,2.0,UPW,7/31/16
1,1,8535032225,212,153.381,153.381,1029.21,1029.21,2.0,NaN,770.0,...,6.0,0.0,1,6,6.0,0.0,NaN,NaN,UPW,6/29/16
2,1,8535032320,1023,22.490,22.490,90.00,90.00,7.0,NaN,2309.0,...,6.0,0.0,2,12,6.0,0.0,NaN,NaN,UPW,6/30/16
3,1,8535032615,944,24856.000,24874.700,6347.61,6347.61,5.0,NaN,4048.0,...,6.0,1.0,1,6,6.0,1.0,NaN,NaN,UPW,8/6/16
4,1,8535032778,660,1252.800,1254.200,1264.76,1847.36,3.0,NaN,770.0,...,6.0,0.0,2,12,6.0,0.0,NaN,NaN,UPW,7/10/16


In [7]:
delinq_df.describe()

/Users/Tultul/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
count,500000.000000,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,450918.000000,13863.000000,450918.000000,...,500000.000000,500000.000000,492286.000000,492286.000000,500000.000000,500000.000000,495400.000000,495400.000000,245979.000000,272256.000000
mean,0.875602,8.447509e+09,650.213808,5382.079277,6073.450843,2684.199749,3468.111061,6.612803,33.840222,2297.278068,...,2.758514,17.948232,6.765067,0.054681,3.689060,23.615604,6.770771,0.046361,6.921380,7.941856
std,0.330035,7.956288e+08,499.094710,9260.163689,10935.203840,4274.347376,5711.741202,9.919921,26.101164,2405.944293,...,2.548578,17.320866,2.001271,0.220220,4.024176,26.420657,2.007622,0.202142,11.531685,12.870727
min,0.000000,7.037000e+09,-48.000000,-99.418667,-99.418667,-9388.400000,-12912.400000,-29.000000,-26.000000,0.000000,...,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,6.000000,0.000000,1.000000,1.000000
25%,1.000000,8.273298e+09,241.000000,42.829500,43.016667,278.910000,300.000000,NaN,NaN,NaN,...,1.000000,6.000000,NaN,NaN,1.000000,6.000000,NaN,NaN,NaN,NaN
50%,1.000000,8.864852e+09,516.000000,1470.000000,1500.000000,1081.855000,1334.200000,NaN,NaN,NaN,...,2.000000,12.000000,NaN,NaN,2.000000,12.000000,NaN,NaN,NaN,NaN
75%,1.000000,9.058449e+09,948.000000,7220.000000,7805.640000,3355.770000,4198.692500,NaN,NaN,NaN,...,4.000000,24.000000,NaN,NaN,5.000000,30.000000,NaN,NaN,NaN,NaN
max,1.000000,9.997999e+09,2438.000000,266539.000000,344937.460000,203129.420000,231865.770000,115.000000,115.000000,86957.000000,...,43.000000,312.000000,12.000000,3.000000,79.000000,474.000000,12.000000,3.000000,171.500000,171.500000


In [8]:
delinq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 36 columns):
label                   500000 non-null int64
msisdn                  500000 non-null int64
aon                     500000 non-null int64
daily_decr30            500000 non-null float64
daily_decr90            500000 non-null float64
rental30                500000 non-null float64
rental90                500000 non-null float64
last_rech_date_ma       450918 non-null float64
last_rech_date_da       13863 non-null float64
last_rech_amt_ma        450918 non-null float64
cnt_ma_rech30           500000 non-null int64
fr_ma_rech30            344761 non-null float64
sumamnt_ma_rech30       500000 non-null int64
medianamnt_ma_rech30    433954 non-null float64
medianmarechprebal30    433954 non-null float64
cnt_ma_rech90           500000 non-null int64
fr_ma_rech90            383581 non-null float64
sumamnt_ma_rech90       500000 non-null int64
medianamnt_ma_rech90    450662 non-null 

In [10]:
delinq_df['label'].value_counts(normalize=True)

1    0.875602
0    0.124398
Name: label, dtype: float64

In [14]:
column_indices = [1,2,3,4,5,6,7,8,9,10]
new_names = ['mobNum', 'ageOnCellNtwrk', 'dailyAmtSpnt30', 'dailyAmtSpnt90', 'avgAcntBal30', 'avgAcntBal90','numDaysTillRchrgM', 'numDaysTillRchrgD', 'amtLastRchrgM', 'numTimesMRchrg30'  ]
old_names = delinq_df.columns[column_indices]
delinq_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [15]:
delinq_df.drop()

,label,mobNum,ageOnCellNtwrk,dailyAmtSpnt30,dailyAmtSpnt90,avgAcntBal30,avgAcntBal90,numDaysTillRchrgM,numDaysTillRchrgD,amtLastRchrgM,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,1,8535032161,1072,8630.000,9191.400,784.48,2007.28,7.0,NaN,773.0,...,6.0,0.0,2,12,6.0,0.0,2.0,2.0,UPW,7/31/16
1,1,8535032225,212,153.381,153.381,1029.21,1029.21,2.0,NaN,770.0,...,6.0,0.0,1,6,6.0,0.0,NaN,NaN,UPW,6/29/16
2,1,8535032320,1023,22.490,22.490,90.00,90.00,7.0,NaN,2309.0,...,6.0,0.0,2,12,6.0,0.0,NaN,NaN,UPW,6/30/16
3,1,8535032615,944,24856.000,24874.700,6347.61,6347.61,5.0,NaN,4048.0,...,6.0,1.0,1,6,6.0,1.0,NaN,NaN,UPW,8/6/16
4,1,8535032778,660,1252.800,1254.200,1264.76,1847.36,3.0,NaN,770.0,...,6.0,0.0,2,12,6.0,0.0,NaN,NaN,UPW,7/10/16


In [ ]:
# column_indices = [1,4,5,6]
# new_names = ['a','b','c','d']
# old_names = df.columns[column_indices]
# df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [13]:
delinq_df['msisdn'].value_counts().unique

<bound method Series.unique of 8868957153    10
8923303528     9
7078965424     9
8864847951     9
7078083013     9
8923594278     9
9084980679     8
7037532871     8
8865084268     8
7037330824     8
8445113313     8
9058667344     8
8534954089     8
8869064708     8
8273325185     8
7037857027     8
8865009028     8
9084943205     8
9084225212     8
7037153544     8
8273176939     8
8535087053     7
9084601322     7
8273886093     7
8869059441     7
9058497820     7
7037212062     7
9084744623     7
7078805515     7
8445099244     7
              ..
9084537792     1
8445682839     1
8868012997     1
8273470405     1
8865907256     1
8868023240     1
9084877272     1
8273859014     1
8445573390     1
8534916014     1
8534905771     1
8923370090     1
8445197563     1
9084425099     1
8273297171     1
7037199664     1
7037649103     1
7078217311     1
8923489173     1
9520655252     1
8532039479     1
8865825689     1
8865833885     1
8865837983     1
9520126588     1
8532847655     1
